# 11785 HW3P2: Automatic Speech Recognition

# README How to run this notebook


The whole jupyter notebook could be separated into several parts, including necessary set up or configuration, dataset configuration, Levenshtein distance calculation, model training and validation and the final submission towards kaggle. 

For the Preliminaries and Libraries import section, we check the type of distributed GPU and kaggle data download and set up. The following train configuration section would set up the account information of wandb and hyperparameter initlization for following training.

The next main part is to build data loader and possible normalization to prepare for the following training. Specifically, cepstral normalizaiton would be applied in this part. 

The next chunk is the core of this notebook. It specifies how I constructed the model. I firstly use the block used in ConvNext to do the embedding, but the expansion was reduced to 2 for calculation efficiency. Then followed by the 4 layer bidirectional lstm layer with the following classifcation layer. Layer initilization is also included in this part. 

Then, it comes to the optimizer set up and Levenshtein distance calculation stuff. With these all set, it finally goes to the training and evaluation part, which would integrate the previous function together and do the training and validating tasks. Additionally, the resume/load model chuncks are used to load the previous saved model for continuous training. 

Finally, I would make prediction on the test datasets and submit to kaggle to check the results.

# Prelimilaries

You will need to install packages for decoding and calculating the Levenshtein distance

In [ ]:
!nvidia-smi

Wed Apr  6 01:06:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

TOKEN = {"username":"yuxuanwucmu","key":"81ded4babfd327efdc7655be369299b5"}

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content/drive/MyDrive/

!kaggle competitions download -c 11-785-s22-hw3p2

!unzip -q /content/drive/MyDrive/competitions/11-785-s22-hw3p2/11-785-s22-hw3p2.zip
!pip install wandb
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
!pip install wandb
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX # We also install a summary package to check our model's forward before training


# Libraries

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import time
from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import csv
import torch.optim as optim
import wandb
from tqdm import tqdm
!pip install timm

import torchaudio
import torchaudio.transforms as T
# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
# PHONEME_MAP is the list that maps the phoneme to a single character. 
# The dataset contains a list of phonemes but you need to map them to their corresponding characters to calculate the Levenshtein Distance
# You final submission should not have the phonemes but the mapped string
# No TODOs in this cell

PHONEME_MAP = [
    " ",
    ".", #SIL
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

PHONEMES = ["", 'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH']

Device:  cuda


# Train configuration

In [ ]:
# batch_size = 256
lr = 2e-3
epochs = 150
batch_size = 64
model = "LSTM_Embedding_layerPlus2_expansion2_block2"
scheduler = "ReduceOnPlateau"
beam_width = "50"
drop_out = 0.5
stride = "stride2"
normalization = "Cepstral"
init ="kaiming"
embedding = "ConvNext"
checkpoints_store_path = "/content/drive/MyDrive/HW3/store_checkpoints/"
create_folder(checkpoints_store_path)

masking = "no"
model_id = f"{model}_{scheduler}_2Cls_dropout{drop_out}_{stride}_{normalization}_{init}_{embedding}_{masking}"
data_path = checkpoints_store_path + model_id
create_folder(data_path)

wandb.init(project="11785-HW3", entity="saltedfish",name=model_id)
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size,
  "drop_out":drop_out
}

wandb: Currently logged in as: 11785_sematicsup (use `wandb login --relogin` to force relogin)


# Dataset and data loading

In [ ]:
# This cell is where your actual TODOs start
# You will need to implement the Dataset class by your own. You may also implement it similar to HW1P2 (dont require context)
# The steps for implementation given below are how we have implemented it.
# However, you are welcomed to do it your own way if it is more comfortable or efficient. 

class LibriSamples(torch.utils.data.Dataset):

    def __init__(self, data_path, partition="train"):  # You can use partition to specify train or dev

        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition + "/transcript/"

        self.X_files = os.listdir(self.X_dir)
        self.Y_files = os.listdir(self.Y_dir)

        # store PHONEMES from phonemes.py inside the class. phonemes.py will be downloaded from kaggle.
        # You may wish to store PHONEMES as a class attribute or a global variable as well.
        self.PHONEMES = PHONEMES

        assert (len(self.X_files) == len(self.Y_files))  # check the length between data and label is the same

        pass

    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, ind):
        X = np.load(
            self.X_dir + self.X_files[ind])  # Each X has a shape of (frames, time step, 13)TODO: Load the mfcc npy file at the specified index ind in the directory
        Y = [self.PHONEMES.index(yy) for yy in
             np.load(self.Y_dir + self.X_files[ind])[1:-1]]  # TODO: Load the corresponding transcripts
        
        X = (X - X.mean(axis=0)) / X.std(axis=0)

        # Remember, the transcripts are a sequence of phonemes. Eg. np.array(['<sos>', 'B', 'IH', 'K', 'SH', 'AA', '<eos>'])
        # You need to convert these into a sequence of Long tensors
        # Tip: You may need to use self.PHONEMES
        # Remember, PHONEMES or PHONEME_MAP do not have '<sos>' or '<eos>' but the transcripts have them.
        # You need to remove '<sos>' and '<eos>' from the trancripts.
        # Inefficient way is to use a for loop for this. Efficient way is to think that '<sos>' occurs at the start and '<eos>' occurs at the end.
        Yy = torch.LongTensor(Y) # TODO: Convert sequence of  phonemes into sequence of Long tensors

        return X, Yy

    def collate_fn(batch):
      
        descending = sorted(batch, key=lambda x: x[0].shape[0], reverse=True) 
        batch_x = [x for x, y in descending]
        batch_y = [y for x, y in descending]


        batch_x_pad = pad_sequence(
            [torch.Tensor(x) for x in batch_x])  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = torch.LongTensor(
            [len(x) for x in batch_x])  # TODO: Get original lengths of the sequence before padding

        batch_y_pad = pad_sequence(batch_y)  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_y = torch.LongTensor(
            [len(y) for y in batch_y])   # TODO: Get original lengths of the sequence before padding

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)



class LibriSamplesTest(torch.utils.data.Dataset):

    def __init__(self, data_path, test_order):  # test_order is the csv similar to what you used in hw1
        file_path = data_path + '/test/' + test_order
        self.X_dir = data_path + "/test/mfcc/"

        test_order_list = self.parse_csv(file_path)  # TODO: open test_order.csv as a list
        self.X = [i for i in test_order_list]  # TODO: Load the npy files from test_order.csv and append into a list
        # You can load the files here or save the paths here and load inside __getitem__ like the previous class

    def __len__(self):
        return len(self.X)

    def __getitem__(self, ind):
        # TODOs: Need to return only X because this is the test dataset
        X = np.load(self.X_dir+self.X[ind][0])
        X = (X - X.mean(axis=0)) / X.std(axis=0)
        return X

    def collate_fn(batch):
        # descending = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
        # batch_x = [x for x in descending]
        batch_x_pad = pad_sequence(
            [torch.Tensor(x) for x in batch])  # TODO: pad the sequence with pad_sequence (already imported)
        lengths_x = torch.LongTensor(
            [len(x) for x in batch])  # TODO: Get original lengths of the sequence before padding

        return batch_x_pad, torch.tensor(lengths_x)

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row)
        return subset[1:]

In [ ]:
root = "/content/hw3p2_student_data/hw3p2_student_data"

train_data = LibriSamples(root, 'train')
val_data = LibriSamples(root, 'dev')
test_data = LibriSamplesTest(root, 'test_order.csv')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True,
                                           collate_fn=LibriSamples.collate_fn)  # TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True,
                                         collate_fn=LibriSamples.collate_fn)  # TODO: Define the val loader. Remember to pass in a parameter (function) for the collate_fn argument
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False,
                                          collate_fn=LibriSamplesTest.collate_fn)  # TODO: Define the test loader. Remember to pass in a parameter (function) for the collate_fn argument

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# Optional
# Test code for checking shapes and return arguments of the train and val loaders
for data in val_loader:
    x, y, lx, ly = data # if you face an error saying "Cannot unpack", then you are not passing the collate_fn argument
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([1746, 64, 13]) torch.Size([209, 64]) torch.Size([64]) torch.Size([64])


# Model Configuration

In [ ]:
# from torch.nn.modules import dropout
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model

class Block(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 expand_ratio, drop_path=0.):
        super().__init__() 

        # change the kernel size to 7 (spatial mix)
        # depth wise convolution
        # 3*3 input padding 1, stride ==1
        self.spatial_mixing = nn.Conv1d(in_channels, in_channels, kernel_size=7, padding=3, stride=1,
                                        groups=in_channels)
        self.norm = nn.BatchNorm1d(in_channels)

        # point wise convolution, increase the channel number, expand ratio =1
        hidden_dim = in_channels * expand_ratio
        self.feature_mixing = nn.Conv1d(in_channels, hidden_dim, kernel_size=1, stride=1, padding=0)
        self.activation = nn.GELU()

        self.bottleneck_channels = nn.Conv1d(hidden_dim, out_channels, kernel_size=1, stride=1, padding=0)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.spatial_mixing(x)
        x = self.norm(x)
        x = self.feature_mixing(x)
        x = self.activation(x)
        x = self.bottleneck_channels(x)
        x = input + self.drop_path(x)

        return x


class Network(nn.Module):

    def __init__(self,drop_out = drop_out):  # You can add any extra arguments as you wish

        super(Network, self).__init__()

        # Embedding layer converts the raw input into features which may (or may not) help the LSTM to learn better
        # For the very low cut-off you dont require an embedding layer. You can pass the input directly to the  LSTM
        self.embedding =nn.ModuleList()
        
        stem = nn.Sequential(nn.Conv1d(13,128,5,stride=2, padding=2),nn.BatchNorm1d(128))
        self.embedding.append(stem)
        
        block = [Block(in_channels=128, out_channels=128, expand_ratio=2,drop_path=0.1)]
        self.embedding.append(nn.Sequential(*block))

        stem2 = nn.Sequential(nn.Conv1d(128,256,3,stride=1, padding=1),nn.BatchNorm1d(256))
        self.embedding.append(stem2)

        self.lstm = nn.LSTM(input_size=256, hidden_size= 512, num_layers=4,bidirectional=True,dropout=0.4)  # TODO: # Create a single layer, uni-directional LSTM with hidden_size = 256
        # Use nn.LSTM() Make sure that you give in the proper arguments as given in https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

        self.classification = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.GELU(),
            nn.Dropout(drop_out),
            
            nn.Linear(2048, 2048),
            nn.GELU(),
            nn.Dropout(drop_out),            

            nn.Linear(2048, 41)
        )

        self._initialize_weights()


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
  

    def forward(self, x, lx):  # TODO: You need to pass at least 1 more parameter apart from self and x
        x = torch.permute(x,(1,2,0))

        # x = T.FrequencyMasking(freq_mask_param=2)(x)

        for i in range(len(self.embedding)):
          x = self.embedding[i](x)

        embedding = x
        lx //=2

        embedding = torch.permute(embedding,(2,0,1))

        # x is returned from the dataloader. So it is assumed to be padded with the help of the collate_fn
        packed_input = pack_padded_sequence(embedding,lx,enforce_sorted=False)  # TODO: Pack the input with pack_padded_sequence. Look at the parameters it requires 
        #enforced_sorted =False

        out1, (out2, out3) = self.lstm(packed_input)  # TODO: Pass packed input to self.lstm
        # As you may see from the LSTM docs, LSTM returns 3 vectors. Which one do you need to pass to the next function?
        # print(out1.shape)
        out, lengths = pad_packed_sequence(out1)  # TODO: Need to 'unpack' the LSTM output using pad_packed_sequence

        out = self.classification(out)  # TODO: Pass unpacked LSTM output to the classification layer
        # out_prob = out.log_softmax(2) # Optional: Do log softmax on the output. Which dimension?

        out_prob = F.log_softmax(out,dim=2) # Optional: Do log softmax on the output. Which dimension?
        # print(out_prob.shape)

        return out_prob, lengths  # TODO: Need to return 2 variables

model = Network().to(device)
print(model)
summary(model, x.to(device), lx) # x and lx are from the previous cell

Network(
  (embedding): ModuleList(
    (0): Sequential(
      (0): Conv1d(13, 128, kernel_size=(5,), stride=(2,), padding=(2,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Block(
        (spatial_mixing): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,), groups=128)
        (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (feature_mixing): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
        (activation): GELU()
        (bottleneck_channels): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
        (drop_path): DropPath()
      )
    )
    (2): Sequential(
      (0): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (lstm): LSTM(256, 512, num_layers=4, dropout=0.4, bidirectional=True)
  (classification): Sequential(
    (0): 

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.0.Conv1d_0,"[13, 128, 5]","[64, 128, 873]",8448.0,7263360.0
1_embedding.0.BatchNorm1d_1,[128],"[64, 128, 873]",256.0,128.0
2_embedding.1.0.Conv1d_spatial_mixing,"[1, 128, 7]","[64, 128, 873]",1024.0,782208.0
3_embedding.1.0.BatchNorm1d_norm,[128],"[64, 128, 873]",256.0,128.0
4_embedding.1.0.Conv1d_feature_mixing,"[128, 256, 1]","[64, 256, 873]",33024.0,28606464.0
5_embedding.1.0.GELU_activation,-,"[64, 256, 873]",NaN,NaN
6_embedding.1.0.Conv1d_bottleneck_channels,"[256, 128, 1]","[64, 128, 873]",32896.0,28606464.0
7_embedding.1.0.DropPath_drop_path,-,"[64, 128, 873]",NaN,NaN
8_embedding.2.Conv1d_0,"[128, 256, 3]","[64, 256, 873]",98560.0,85819392.0


# Optimizer Configuration

In [ ]:
criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= lr) # TODO: Adam works well with LSTM (use lr = 2e-3)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4,factor=0.2, mode="min")

decoder = CTCBeamDecoder(PHONEME_MAP,log_probs_input=True, beam_width=1) # TODO: Intialize the CTC beam decoder
decoder_test= CTCBeamDecoder(PHONEME_MAP,log_probs_input=True, beam_width=50)
scaler = torch.cuda.amp.GradScaler()


# Levenshtein distance calculation


In [ ]:
def calculate_levenshtein(h, y, ly,lh,decoder, PHONEME_MAP):

    # h - ouput from the model. Probability distributions at each time step 
    # y - target output sequence - sequence of Long tensors
    # lh, ly - Lengths of output and target
    # decoder - decoder object which was initialized in the previous cell
    # PHONEME_MAP - maps output to a character to find the Levenshtein distance

    h = h.permute(1,0,2) # potential problems (batch first model) 

    # TODO: call the decoder's decode method and get beam_results and out_len (Read the docs about the decode method's outputs)
    # Input to the decode method will be h and its lengths lh 
    # You need to pass lh for the 'seq_lens' parameter. This is not explicitly mentioned in the git repo of ctcdecode.
    beam_results, _, _, out_lens = decoder.decode(h,lh)

    batch_size = len(beam_results)

    dist = 0

    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = beam_results[i][0]# TODO: Get the output as a sequence of numbers from beam_results

        # Remember that h is padded to the max sequence length and lh contains lengths of individual sequences
        # Same goes for beam_results and out_lens
        # You do not require the padded portion of beam_results - you need to slice it with out_lens 
        # If it is confusing, print out the shapes of all the variables and try to understand

        h_string = "".join(PHONEME_MAP[x] for x in h_sliced[0:out_lens[i][0]]) # TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string

        y_sliced = y[i] # TODO: Do the same for y - slice off the padding with ly

        y_string = "".join(PHONEME_MAP[i] for i in y_sliced[0:ly[i]]) # TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string
        
        dist += Levenshtein.distance(h_string, y_string)

    dist/=batch_size

    return dist

# Training and Evaluation 

In [ ]:
torch.cuda.empty_cache()

for epoch in range(1,epochs+1):
    start_time = time.time()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    running_loss = 0.0
    totalSampleCnt = 0.0
    
    for batch_idx, (data, target, data_lens, target_lens) in enumerate(train_loader):
        optimizer.zero_grad()
        target = target.permute(1,0)
        data, target= data.cuda(), target.cuda()
        
        with torch.cuda.amp.autocast():  
            output, output_lens = model(data,data_lens)
            loss = criterion(output,target, output_lens,target_lens)

        running_loss+=loss.item()
        totalSampleCnt += len(data)

        loss_per_sample = running_loss / len(val_loader)

        batch_bar.set_postfix(
                loss="{:.04f}".format(float(loss_per_sample)),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        lr_wb = round(float(optimizer.param_groups[0]['lr']),8)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        batch_bar.update()
    batch_bar.close()


    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        running_charErr = 0.0
        totalSampleCnt = 0
        
        for batch_idx, (data, target, data_lens, target_lens) in enumerate(val_loader):
            target = target.permute(1,0)
            data, target = data.cuda(), target.cuda()
            output, output_lens = model(data, data_lens)    
            loss = criterion(output,target, output_lens,target_lens)

            running_loss+=loss.item()
            totalSampleCnt += len(data)

            dist = calculate_levenshtein(output,target,target_lens,output_lens,decoder,PHONEME_MAP)
            running_charErr += dist
        
        loss_per_sample = running_loss / len(val_loader)
        dist_per_sample = running_charErr/len(val_loader)
        
        scheduler.step(dist_per_sample)

        print("epoch ",epoch,"loss is: ",loss_per_sample, " Levenshtein is: ",dist_per_sample)
        
        wandb.log({"loss": loss,"Levenshtein":dist_per_sample,"lr":lr_wb,"epoch":epoch})
    
    end_time = time.time()
    print("Whole Epoch takes: ", (end_time - start_time)/60,"minutes to train")

    path = data_path+f"/model_epoch_{epoch}.ckpt"

    if epoch % 5==0:
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(), 
          }, path)
            

# Resume model

In [ ]:
path = "/content/drive/MyDrive/HW3/store_checkpoints/LSTM_Embedding_layerPlus2_expansion2_block2_ReduceOnPlateau_2Cls_dropout0.5_stride2_Cepstral_kaiming_ConvNext_no/model_epoch_98.ckpt"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
torch.cuda.empty_cache()

for epoch in range(107,epochs+1):
    start_time = time.time()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    running_loss = 0.0
    totalSampleCnt = 0.0
    
    for batch_idx, (data, target, data_lens, target_lens) in enumerate(train_loader):
        optimizer.zero_grad()
        target = target.permute(1,0)
        data, target= data.cuda(), target.cuda()
        
        with torch.cuda.amp.autocast():  
            output, output_lens = model(data,data_lens)
            loss = criterion(output,target, output_lens,target_lens)

        running_loss+=loss.item()
        totalSampleCnt += len(data)

        loss_per_sample = running_loss / len(val_loader)

        batch_bar.set_postfix(
                loss="{:.04f}".format(float(loss_per_sample)),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        lr_wb = round(float(optimizer.param_groups[0]['lr']),8)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        batch_bar.update()
    batch_bar.close()


    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        running_charErr = 0.0
        totalSampleCnt = 0
        
        for batch_idx, (data, target, data_lens, target_lens) in enumerate(val_loader):
            target = target.permute(1,0)
            data, target = data.cuda(), target.cuda()
            output, output_lens = model(data, data_lens)    
            loss = criterion(output,target, output_lens,target_lens)

            running_loss+=loss.item()
            totalSampleCnt += len(data)

            dist = calculate_levenshtein(output,target,target_lens,output_lens,decoder,PHONEME_MAP)
            running_charErr += dist
        
        loss_per_sample = running_loss / len(val_loader)
        dist_per_sample = running_charErr/len(val_loader)
        
        scheduler.step(dist_per_sample)

        print("epoch ",epoch,"loss is: ",loss_per_sample, " Levenshtein is: ",dist_per_sample)
        
        wandb.log({"loss": loss,"Levenshtein":dist_per_sample,"lr":lr_wb,"epoch":epoch})
    
    end_time = time.time()
    print("Whole Epoch takes: ", (end_time - start_time)/60,"minutes to train")

    path = data_path+f"/model_epoch_{epoch}.ckpt"

    if epoch % 2==0:
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(), 
          }, path)
            

# Load Model

In [ ]:
decoder_test= CTCBeamDecoder(PHONEME_MAP,log_probs_input=True, beam_width=50)
path = "/content/drive/MyDrive/HW3/store_checkpoints/LSTM_Embedding_layerPlus2_expansion2_block2_ReduceOnPlateau_2Cls_dropout0.5_stride2_Cepstral_kaiming_ConvNext_no/model_epoch_116.ckpt"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Predict

In [ ]:
model.eval()
res = []

for batch_idx,(data, data_lens) in enumerate(test_loader):
    data = data.cuda()
    model = model.cuda()
    output, output_lens = model(data,data_lens)
    output = output.permute(1,0,2)
    beam_results, _, _, out_lens = decoder_test.decode(output,output_lens)

    batch_size = len(beam_results)

    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = beam_results[i][0]# TODO: Get the output as a sequence of numbers from beam_results
        h_string = "".join(PHONEME_MAP[x] for x in h_sliced[0:out_lens[i][0]]) # TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string

        # print(h_string)

        res.append(h_string)

# res

In [ ]:
import pandas as pd
id = np.array(range(len(res)))
predictions = res
df = pd.DataFrame({"id":id,"predictions":predictions})
df.to_csv("submission.csv",index=False)

!kaggle competitions submit -c 11-785-s22-hw3p2 -f submission.csv -m "Message"

100% 214k/214k [00:01<00:00, 117kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)

# Submit to kaggle (TODO)

In [ ]:
# TODO: Write your model evaluation code for the test dataset
# You can write your own code or use from the previous homewoks' stater notebooks
# You can't calculate loss here. Why?

In [ ]:
# TODO: Generate the csv file